**30. La 3-upla de palabras más común en los comentarios de los logs (⭐⭐)**


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

Para este ejercicio voy a utilizar el csv **logs.**

In [ ]:
downloaded = drive.CreateFile({'id': '1nIzHWsLqzJWq9XuKEweSwMCPEGkysoGB'})
downloaded.GetContentFile('logs.csv')

Como el csv es muy grande, solo me voy a quedar con algunas pocas columnas para ahorrar memoria

In [ ]:
logs = pd.read_csv('logs.csv', encoding='latin-1' , usecols=["item_id", "contributor_username", "comment"],\
                            dtype={'contributor_username':'category'})
logs.head()

,item_id,contributor_username,comment
0,1,Sabbut,el contenido era: 'Si aprendÃ©s Pensamiento Ci...
1,2,Sabbut,el contenido era: '{{destruir}}en algun lugar ...
2,3,Sabbut,el contenido era: '#REDIRECT [[Flash]]'
3,4,Sanbec,Sin informaciÃ³n de origen ni de licencia. Hay...
4,5,Sanbec,Sustituida por [[commons:Image:Heckert_GNU.png]]


Veo si hay NaNs en la columna de comment:

In [ ]:
logs['comment'].hasnans

True

Hago una limpieza de NaNs:

In [ ]:
logs = logs.dropna()
logs['comment'].hasnans

False

In [ ]:
logs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5208787 entries, 0 to 12188944
Data columns (total 3 columns):
 #   Column                Dtype   
---  ------                -----   
 0   item_id               int64   
 1   contributor_username  category
 2   comment               object  
dtypes: category(1), int64(1), object(1)
memory usage: 346.3+ MB


Para encontrar la 3-upla más común voy a utilizar n-grams a partir de la función CountVectorizer.

Son más de 5.2 millones de comentarios, utilizandolos todos tarda muchisimo, lo deje varias horas y seguia.

Lo reduci a medio millón lo deje más de 2 horas y seguía...

Asi que lo voy a realizar con 100.000 para hacerlo más rápido, pero quiero aclarar que esto funciona con el total de datos pero tarda demasiado


In [ ]:
word_vectorizer = CountVectorizer(ngram_range=(3,3), analyzer='word')

sparse_matrix = word_vectorizer.fit_transform(logs[:100000]['comment'])

frequencies = sum(sparse_matrix).data

pd.DataFrame(frequencies, index=word_vectorizer.get_feature_names(), columns=['comment']).sort_values('comment',ascending=False)

,comment
los sitemas de,33180
eric bruno borgman,28147
era no leagues,27480
era no lo,27480
los sims ya,11155
...,...
era mejorar las,1
era mejorar la,1
era mejorar garganta,1
era mejorar es,1


Podemos ver que la 3-upla más común entre esta porción de datos es: **los sistemas de**

Si vemos otra porción de 100.000:

In [ ]:
word_vectorizer = CountVectorizer(ngram_range=(3,3), analyzer='word')

sparse_matrix = word_vectorizer.fit_transform(logs[100000:200000]['comment'])

frequencies = sum(sparse_matrix).data

pd.DataFrame(frequencies, index=word_vectorizer.get_feature_names(), columns=['comment']).sort_values('comment',ascending=False)

,comment
nito nora discusiã³n,54911
niten ichy niten,47270
niten ichy discusiã³n,47270
ninoconc discusiã³n special,19328
talk julius47 discusiã³n,7641
...,...
freewebs com blaxdas,1
freez discusiã³n special,1
freez titan freez,1
freez user talk,1


En este caso la 3-upla cambia.